In [ ]:
import dash
from dash.dependencies import Input, Output
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import ast

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

#Import datasets
position_clusters = pd.read_csv('Stats\\cb\\player_classifications.csv', header=0)
position_details = pd.read_csv('Stats\\cb\\data.csv', header=0)
important_features = pd.read_csv('Stats\\important_features.csv', header=0, index_col = 0)
player_details = pd.read_csv('created_data\\player_details.csv', header=0, encoding = 'latin1')

#Get components for starting position 
feature_columns = [f for f in list(important_features) if "CB" in f]
display_features = important_features[feature_columns]

#Define positions
position = ["GK", "CB", "FB", "DM", "CM", "AM","Winger", "Forward"]

In [ ]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = 

    html.Div([
        
        html.Div([
            #Position Selector    
            html.Div(id='select_pos', children = "Select Position", 
                     style = {'margin': 'auto', 'width': '50%', 'text-align':'center', 
                              'font-size' :'200%', 'font-weight':'bold'}),
            dcc.Dropdown(id='position',
                         options=[{'label': i, 'value': i} for i in position],
                         value='CB'),
            html.H5(' ', className='spacing'),
            
            #League Selector
            html.Div(children = "Select First Player:", 
                     style = {'margin': 'auto','float':'left', 'width': '100%', 
                              'font-size' :'150%', 'margin-top':'10px', 'font-weight':'bold'}),            
            html.Div([             
                    html.P('Select Leaue: ', className='spacing'),
                    dcc.Dropdown(id='league_1',
                         options=[{'label': i, 'value': i} for i in player_details["league"].drop_duplicates().dropna()],
                         value='English Premier League')], 
                    style={'display': 'inline-block', 'width':'45%', 'height':'100%', 'float':'left', 'padding': '10px'}),
                                 
            #Team Selector                      
            html.Div([
                    html.P('Select Team: ', className='spacing'),              
                    dcc.Dropdown(id='team_1',
                         value='Manchester City')], 
                    style={'display': 'inline-block', 'width':'45%', 'height':'100%', 'float':'left', 'padding': '10px'}),
              
            #Player Selector
            dash_table.DataTable(id='datatable',
                columns=[{"name": i, "id": i} for i in position_details.loc[:,["full_name"]].columns],
                data=position_details.to_dict("rows"),  
                pagination_settings={'current_page': 0,'page_size': 5},
                pagination_mode='be',
                row_selectable="single",
                selected_rows=[]),
                                 
            #League Selector 2
   
            html.Div(children = "Select Second Player:", style = {'margin': 'auto','float':'left', 'width': '100%', 'font-size' :'150%', 'margin-top':'10px', 'font-weight':'bold'}),

            html.Div([             
                    html.P('Select Leaue: ', className='spacing'),
                    dcc.Dropdown(id='league_2',
                         options=[{'label': i, 'value': i} for i in player_details["league"].drop_duplicates().dropna()],
                         value='English Premier League')], 
                    style={'display': 'inline-block', 'width':'45%', 'height':'100%', 'float':'left', 'padding': '10px'}),
                                 
            #Team Selector 2                      
            html.Div([
                    html.P('Select Team: ', className='spacing'),              
                    dcc.Dropdown(id='team_2',
                         value='Manchester City')], 
                    style={'display': 'inline-block', 'width':'45%', 'height':'100%', 'float':'left', 'padding': '10px'}),                     
            dash_table.DataTable(id='datatable_2',
                columns=[{"name": i, "id": i} for i in position_details.loc[:,["full_name"]].columns],
                data=position_details.to_dict("rows"),  
                pagination_settings={'current_page': 0,'page_size': 5},
                pagination_mode='be',
                row_selectable="single",
                selected_rows=[]),
           ], style={'display': 'inline-block', 'width':'30%', 'height':'100%', 'float':'left', 'padding': '10px', 'text_align':'center'}),

        html.Div([
                html.Div(id='selected_player_2', style = {'margin': 'auto', 'width': '70%', 'text-align':'center', 'font-size':'200%', 'font-weight':'bold'}),
                dcc.Graph( id= 'radar_plot',
                    figure=go.Figure(
                            data = [go.Scatterpolar(
                              r = position_clusters.iloc[0,1:len(position_clusters.columns)-2].tolist() + position_clusters.iloc[0,1:len(position_clusters.columns)-2].tolist()[:1],
                              theta = position_clusters.columns[1:len(position_clusters.columns)-2].tolist() + position_clusters.columns[1:len(position_clusters.columns)-2].tolist()[:1],
                              fill = 'toself',
                              name = player_details.iloc[0,3]
                              
                            ),
                            go.Scatterpolar(
                              r = position_clusters.iloc[1,1:len(position_clusters.columns)-2].tolist() + position_clusters.iloc[1,1:len(position_clusters.columns)-2].tolist()[:1],
                              theta = position_clusters.columns[1:len(position_clusters.columns)-2].tolist() + position_clusters.columns[1:len(position_clusters.columns)-2].tolist()[:1],
                              fill = 'toself',
                              name = player_details.iloc[1,3]
                            )],
                            
                            layout = go.Layout(
                              polar = dict(
                                radialaxis = dict(
                                  visible = True,
                                  range = [0, 1]
                                )
                              ),
                              showlegend = True,
                              legend=dict(x = 0, y = 1, font=dict(
                                    size=20,
                                ))
                            )
                            
                            )
                    ),
                html.Div([dash_table.DataTable(id='features',
                    columns=[{"name": i, "id": i} for i in display_features.columns],
                    data=display_features.to_dict("rows"))],style={'padding-top':'10px'} )
                ], style={'display': 'inline-block','float':'right', 'width':'65%', 'height':'100%', 'padding': '10px'})
    ])

#Callbacks 
#DIV1
@app.callback(
    Output(component_id='datatable', component_property='data'),
    [Input(component_id='position', component_property='value'),
    Input(component_id='team_1', component_property='value'),
    Input(component_id = 'datatable', component_property = 'pagination_settings')]
)
def update_players(input_value, team_value, page):
    player_array = pd.read_csv('C:\\Users\\Tom\\Documents\\optaForum\\data\\created_data\\Stats\\'+ input_value +'\\data.csv', header=0)
    player_array = player_array[player_array['team_name'] == team_value]
    current_page = page["current_page"]
    size = page["page_size"]
    paged_array = player_array.iloc[current_page*size:(current_page + 1)*size].to_dict('rows')
    return paged_array

@app.callback(
    Output('selected_player_2', "children"),
    [Input('datatable', "derived_virtual_data"),
     Input('datatable', "derived_virtual_selected_rows"),
     Input('datatable_2', "derived_virtual_data"),
     Input('datatable_2', "derived_virtual_selected_rows")])
def update_graph(rows, derived_virtual_selected_rows, rows2, derived_virtual_selected_rows2 ):
    dff = pd.DataFrame(rows)
    dff2 = pd.DataFrame(rows2)       
       
    player_1 = dff.loc[derived_virtual_selected_rows,"full_name"].iloc[0]
    player_2 = dff2.loc[derived_virtual_selected_rows2,"full_name"].iloc[0]      
      
    return player_1 + " vs " + player_2

#DIV2

@app.callback(
    Output(component_id='datatable_2', component_property='data'),
    [Input(component_id='position', component_property='value'),
     Input(component_id='team_2', component_property='value'),
    Input(component_id = 'datatable_2', component_property = 'pagination_settings')]
)
def update_players_2(input_value, team_value,page):
    player_array = pd.read_csv('C:\\Users\\Tom\\Documents\\optaForum\\data\\created_data\\Stats\\'+ input_value +'\\data.csv', header=0)
    player_array = player_array[player_array['team_name'] == team_value]
    current_page = page["current_page"]
    size = page["page_size"]
    paged_array = player_array.iloc[current_page*size:(current_page + 1)*size].to_dict('rows')
    return paged_array



@app.callback(
    Output('radar_plot', "figure"),
    [Input('datatable', "derived_virtual_data"),
     Input('datatable', "derived_virtual_selected_rows"),
     Input('datatable_2', "derived_virtual_data"),
     Input('datatable_2', "derived_virtual_selected_rows"),
     Input(component_id='position', component_property='value')])
def update_radar(rows, derived_virtual_selected_rows, rows2, derived_virtual_selected_rows2, input_value):
    
    position_clusters = pd.read_csv('C:\\Users\\Tom\\Documents\\optaForum\\data\\created_data\\Stats\\'+ input_value +'\\player_classifications.csv', header=0)
    dff = pd.DataFrame(rows)
    player_id = dff.loc[derived_virtual_selected_rows,"player_id"].iloc[0]
    player_stats = position_clusters[position_clusters["player_id"] == player_id]
    player_name = dff.loc[derived_virtual_selected_rows,"full_name"].iloc[0]
    
    dff2 = pd.DataFrame(rows2)
    player_id2 = dff2.loc[derived_virtual_selected_rows2,"player_id"].iloc[0]
    player_stats2 = position_clusters[position_clusters["player_id"] == player_id2]
    player_name2 = dff2.loc[derived_virtual_selected_rows2,"full_name"].iloc[0]
    
    return go.Figure(
                            data = [
                                    go.Scatterpolar(
                                      r = player_stats.iloc[0,1:len(player_stats.columns)-2].tolist() + player_stats.iloc[0,1:len(player_stats.columns)-2].tolist()[:1],
                                      theta = player_stats.columns[1:len(player_stats.columns)-2].tolist() + player_stats.columns[1:len(player_stats.columns)-2].tolist()[:1],
                                      fill = 'toself',
                                      name = player_name
                                    ),
    
                                  go.Scatterpolar(
                                      r = player_stats2.iloc[0,1:len(player_stats2.columns)-2].tolist() + player_stats2.iloc[0,1:len(player_stats2.columns)-2].tolist()[:1],
                                      theta = player_stats2.columns[1:len(player_stats2.columns)-2].tolist() + player_stats2.columns[1:len(player_stats2.columns)-2].tolist()[:1],
                                      fill = 'toself',
                                      name = player_name2
                                      )
                            ],
                            
                            layout = go.Layout(
                              polar = dict(
                                radialaxis = dict(
                                  visible = True,
                                  range = [0, 1]
                                )
                                
                              ),
                              showlegend = True,
                              legend=dict(x = 0, y = 1, font=dict(
                                    size=20,
                                ))
                            )
                            
                            )

@app.callback(
    Output(component_id='features', component_property='columns'),
    [Input(component_id='position', component_property='value'),
    Input(component_id = 'datatable_2', component_property = 'pagination_settings')]
)
def update_feature_columns(input_value,page):
    feature_columns = [f for f in list(important_combined) if input_value + " - " in f]
    #display_features = important_combined[feature_columns]
    return [{"name": i, "id": i} for i in feature_columns]

@app.callback(
    Output(component_id='features', component_property='data'),
    [Input(component_id='position', component_property='value'),
    Input(component_id = 'datatable_2', component_property = 'pagination_settings')]
)
def update_feature_data(input_value,page):
    feature_columns = [f for f in list(important_combined) if input_value + " - " in f]
    display_features = important_combined[feature_columns]
    return display_features.to_dict("rows")

#team_filter
@app.callback(
    Output(component_id='team_1', component_property='options'),
    [Input(component_id='league_1', component_property='value')]
    )
def update_team_1(input_value):
    team_array = player_details[ player_details['league'] == input_value]
    return [{'label': i, 'value': i} for i in team_array["team_name"].drop_duplicates().dropna()]

@app.callback(
    Output(component_id='team_2', component_property='options'),
    [Input(component_id='league_2', component_property='value')]
    )
def update_team_2(input_value):
    team_array = player_details[ player_details['league'] == input_value]
    return [{'label': i, 'value': i} for i in team_array["team_name"].drop_duplicates().dropna()]



if __name__ == '__main__':
    app.run_server(debug=True,port=8051)